In [2]:
import numpy as np
import pandas as pd

In [11]:
def heston_bates_charfunc(u, params):
    S0    = params["S0"]
    K     = params["K"]
    r     = params["r"]
    q     = params["q"]
    λ     = params["lambda"]
    μJ    = params["muJ"]
    δJ    = params["deltaJ"]
    v0    = params["v0"]
    kappa = params["kappa"]
    theta = params["theta"]
    sigma = params["sigma"]
    rho   = params["rho"]
    T     = params["T"]

    iu = 1j * u

    # —— Heston core (unchanged) ——
    alpha = -0.5 * (u**2 + iu)
    beta  = kappa - rho * sigma * iu
    gamma = 0.5 * sigma**2
    d     = np.sqrt(beta**2 - 4.0 * alpha * gamma)
    g     = (beta - d) / (beta + d)
    exp_dT = np.exp(-d * T)
    C = ((kappa*theta/gamma)*((beta-d)*T - 2*np.log((1 - g*exp_dT)/(1 - g))))
    D = ((beta-d)/gamma) * ((1-exp_dT)/(1-g*exp_dT))


    # —— Adjust drift for jumps ——
    r_tilde = r - q - λ*(np.exp(μJ + 0.5*δJ**2) - 1)
    drift  = r_tilde * iu * T

    # —— Bates jump‐part ——
    jump_cf   = np.exp(iu*μJ - 0.5*(u**2)*δJ**2)
    jump_term = λ * T * (jump_cf - 1)

    C = drift + C + jump_term

    return np.exp(C + D * v0 + iu * np.log(S0 * np.exp(-q * T)))

def heston_price_call_fft_bates(params, N=10000, U_max=1000):
    S0 = params["S0"]
    r  = params["r"]
    q  = params["q"]
    T  = params["T"]
    K  = params["K"]

    if N % 2 == 1:
        N += 1

    u = np.linspace(1e-10, U_max, N + 1)
    du = u[1] - u[0]

    lnK = np.log(K)
    phi_u     = heston_bates_charfunc(u, params)
    phi_u_im1 = heston_bates_charfunc(u - 1j, params)
    phi_im1   = heston_bates_charfunc(-1j, params)

    integrand_P1 = np.real(np.exp(-1j * u * lnK) * phi_u_im1 / (1j * u * phi_im1))
    integrand_P2 = np.real(np.exp(-1j * u * lnK) * phi_u / (1j * u))

    weights = np.ones(N + 1)
    weights[1:-1:2] = 4.0
    weights[2:-2:2] = 2.0

    P1 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand_P1)
    P2 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand_P2)

    call = S0 * np.exp(-q * T) * P1 - K * np.exp(-r * T) * P2
    put  = K * np.exp(-r * T) * (1 - P2) - S0 * np.exp(-q * T) * (1 - P1)

    return call, put


In [14]:
params = {
    "S0": 3795.74,
    "K" : 3795.74,
    "r":  0.014,
    "q": 0.015,
    "v0": 0.0112,
    "lambda": 0.1,
    "muJ": -0.1,
    "deltaJ": 0.2,
    "kappa": 0.2183,
    "theta": 0.0601,
    "sigma": 0.3171,
    "rho": -0.5250,
    "T": 1.0
}

call,put = heston_price_call_fft_bates(params, N=10000, U_max=1000)
print(f"Call Price: {call}")
print(f"Put Price: {put}")

Call Price: 255.0806979509839
Put Price: 258.82179698236973
